# Quick Start

See the API for more detailed information, examples, formulas, and references for each function.

In [1]:
import numpy as np
import xarray as xr
import xskillscore as xs

Here, we generate some sample gridded data. Our data has three time steps, and a 4x5 latitude/longitude grid. `obs` replicates some verification data and `fct` some forecast (e.g. from a statistical or dynamical model).

In [2]:
obs = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.arange(4),
           np.arange(5),
       ],
       dims=["time", "lat", "lon"],
   )
fct = obs.copy()
fct.values = np.random.rand(3, 4, 5)

## Deterministic Metrics

`xskillscore` offers a suite of correlation-based and distance-based deterministic metrics.

### Correlation-Based 

* Pearson Correlation (`pearson_r`)
* Pearson Correlation p value (`pearson_r_p_value`)
* Pearson Correlation effective p value (`pearson_r_eff_p_value`)
* Spearman Correlation (`spearman_r`)
* Spearman Correlation p value (`spearman_r_p_value`)
* Spearman Correlation effective p value (`spearman_r_eff_p_value`)
* Effective Sample Size (`effective_sample_size`)
* Coefficient of Determination (`r2`)

### Distance-Based

* Root Mean Squared Error (`rmse`)
* Mean Squared Error (`mse`)
* Mean Absolute Error (`mae`)
* Median Absolute Error (`median_absolute_error`)
* Symmetric Mean Absolute Percentage Error (`smape`)
* Mean Absolute Percentage Error (`mape`)

Calling the functions is very straight-forward. All deterministic functions take the form `func(a, b, dim, **kwargs)`. **Notice that the original dataset is reduced by the dimension passed.** I.e., since we passed `time` as the dimension here, we are returned an object with dimensions `(lat, lon)`.

In [3]:
r = xs.pearson_r(obs, fct, "time")
print(r)

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.60757059, -0.90716985, -0.99538423,  0.91753774, -0.9564352 ],
       [-0.3048408 ,  0.62504358, -0.40671679, -0.9501542 , -0.75568284],
       [-0.30422357,  0.99917348, -0.86242569,  0.99121445,  0.9950846 ],
       [ 0.99943239,  0.90966433,  0.18115566, -0.345191  , -0.46966954]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [4]:
p = xs.pearson_r_p_value(obs, fct, "time")
print(p)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.58428836, 0.27647634, 0.06119057, 0.26034744, 0.18860465],
       [0.80279348, 0.57016238, 0.73334717, 0.20185062, 0.45461032],
       [0.80320602, 0.02588523, 0.33788845, 0.08444968, 0.06314696],
       [0.0214507 , 0.27267772, 0.88403243, 0.77562827, 0.68874613]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also specify multiple axes for deterministic metrics. Here, we apply it over the latitude and longitude dimension (a pattern correlation).

In [5]:
r = xs.pearson_r(obs, fct, ["lat", "lon"])
print(r)

<xarray.DataArray (time: 3)>
array([-0.18249252, -0.06656047, -0.04829026])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


All deterministic metrics except for `pearson_r_eff_p_value`, `spearman_r_eff_p_value`, and `effective_sample_size` can take the kwarg `weights=...`. `weights` should be a DataArray of the size of the reduced dimension (e.g., if time is being reduced it should be of length 3 in our example).

Weighting is a common practice when working with observations and model simulations of the Earth system. When working with rectilinear grids, one can weight the data by the cosine of the latitude, which is maximum at the equator and minimum at the poles (as in the below example). More complicated model grids tend to be accompanied by a cell area varaible, which could also be passed into this function.

In [6]:
obs2 = xr.DataArray(
       np.random.rand(3, 180, 360),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.linspace(-89.5, 89.5, 180),
           np.linspace(-179.5, 179.5, 360),
       ],
       dims=["time", "lat", "lon"],
    )
fct2 = obs2.copy()
fct2.values = np.random.rand(3, 180, 360)

In [7]:
# make weights as cosine of the latitude and broadcast
weights = np.cos(np.deg2rad(obs2.lat))
_, weights = xr.broadcast(obs2, weights)

# Remove the time dimension from weights
weights = weights.isel(time=0)

In [8]:
r_weighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=weights)
print(r_weighted)

<xarray.DataArray (time: 3)>
array([-0.00529634,  0.00427004, -0.00200837])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [9]:
r_unweighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=None)
print(r_unweighted)

<xarray.DataArray (time: 3)>
array([-0.005734  ,  0.00195357, -0.00219713])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


You can also pass the optional boolean kwarg `skipna=...`. If `True`, ignore any NaNs (pairwise) in `a` and `b` when computing the result. If `False`, return NaNs anywhere there are pairwise NaNs.

In [10]:
obs_with_nans = obs.where(obs.lat > 1)
fct_with_nans = fct.where(fct.lat > 1)
print(obs_with_nans)

<xarray.DataArray (time: 3, lat: 4, lon: 5)>
array([[[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.49535755, 0.43770543, 0.21474886, 0.74063281, 0.90344321],
        [0.39788125, 0.26773374, 0.95455719, 0.73896065, 0.35322886]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.33881785, 0.08226066, 0.03739344, 0.82474385, 0.69740342],
        [0.00324568, 0.53281184, 0.84237521, 0.96839563, 0.75316859]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.60168374, 0.30334218, 0.45654646, 0.19456569, 0.540337  ],
        [0.89271105, 0.01077949, 0.71463065, 0.31552161, 0.67672927]]])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00
  * lat      (lat) int64 

In [11]:
mae_with_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=True)
print(mae_with_skipna)

<xarray.DataArray (time: 3)>
array([0.35732126, 0.30569198, 0.34095427])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [12]:
mae_without_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=False)
print(mae_without_skipna)

<xarray.DataArray (time: 3)>
array([nan, nan, nan])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Probabilistic Metrics

`xskillscore` offers a suite of probabilistic metrics, mostly ported from `properscoring`.

* Brier Score (`brier_score`)
* Continuous Ranked Probability Score with the ensemble distribution (`crps_ensemble`)
* Continuous Ranked Probability Score with a Gaussian distribution (`crps_gaussian`)
* Continuous Ranked Probability Score with numerical integration of the normal distribution (`crps_quadrature`)
* Brier scores of an ensemble for exceeding given thresholds (`threshold_brier_score`)


We now create some data with an ensemble member dimension. In this case, we envision an ensemble forecast with multiple members to validate against our theoretical observations.

In [13]:
obs3 = xr.DataArray(
       np.random.rand(4, 5),
       coords=[np.arange(4), np.arange(5)],
       dims=["lat", "lon"]
   )
fct3 = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[np.arange(3), np.arange(4), np.arange(5)],
       dims=["member", "lat", "lon"],
   )

Continuous Ranked Probability Score with the ensemble distribution.

In [14]:
crps_ensemble = xs.crps_ensemble(obs3, fct3)
print(crps_ensemble)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.15553996, 0.21717016, 0.28250705, 0.02925143, 0.39547273],
       [0.24511768, 0.5109745 , 0.09432956, 0.29830276, 0.16252765],
       [0.06187008, 0.2863955 , 0.09767171, 0.08919699, 0.14553971],
       [0.40126171, 0.03885742, 0.19205888, 0.1005629 , 0.11252024]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS with a Gaussian distribution requires two parameters: $\mu$ and $\sigma$ from the forecast distribution. Here, we just use the ensemble mean and ensemble spread.

In [15]:
crps_gaussian = xs.crps_gaussian(obs3, fct3.mean("member"), fct3.std("member"))
print(crps_gaussian)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.15366326, 0.21478066, 0.27901015, 0.03052345, 0.39069841],
       [0.2384425 , 0.50751335, 0.08083851, 0.30602978, 0.15940914],
       [0.06582983, 0.28357811, 0.07593713, 0.08924322, 0.18097999],
       [0.39526181, 0.02966916, 0.16445858, 0.055777  , 0.11513333]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS quadrature metric requires a callable distribution function. Here we use `norm` from `scipy.stats`.

In [17]:
from scipy.stats import norm
crps_quadrature = xs.crps_quadrature(obs3, norm)
print(crps_quadrature)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.25249613, 0.56778612, 0.54441296, 0.26976489, 0.5952479 ],
       [0.352512  , 0.55132442, 0.39940837, 0.52103694, 0.33877407],
       [0.27890125, 0.36797096, 0.34716667, 0.3182155 , 0.23832836],
       [0.23409808, 0.4779008 , 0.25681028, 0.27633459, 0.30941088]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


We can also use a threshold Brier Score, to score hits over a certain threshold.

In [18]:
threshold_brier_score = xs.threshold_brier_score(obs3, fct3, 0.7)
print(threshold_brier_score)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.        , 0.44444444, 1.        , 0.        , 0.44444444],
       [0.        , 1.        , 0.11111111, 0.44444444, 0.        ],
       [0.        , 0.        , 0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111, 0.        , 0.        ]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


## Accessors

You can also use `xskillscore` as a method of your `xarray` Dataset.

In [19]:
ds = xr.Dataset()
ds["obs_var"] = obs
ds["fct_var"] = fct

In the case that your Dataset contains both your observation and forecast variable, just pass them as strings into the function.

In [20]:
ds.xs.pearson_r("obs_var", "fct_var", "time")

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.60757059, -0.90716985, -0.99538423,  0.91753774, -0.9564352 ],
       [-0.3048408 ,  0.62504358, -0.40671679, -0.9501542 , -0.75568284],
       [-0.30422357,  0.99917348, -0.86242569,  0.99121445,  0.9950846 ],
       [ 0.99943239,  0.90966433,  0.18115566, -0.345191  , -0.46966954]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4

You can also pass in a separate Dataset that contains your observations or forecast variable.

In [21]:
ds = ds.drop_vars("fct_var")
ds.xs.pearson_r("obs_var", fct, "time")

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.60757059, -0.90716985, -0.99538423,  0.91753774, -0.9564352 ],
       [-0.3048408 ,  0.62504358, -0.40671679, -0.9501542 , -0.75568284],
       [-0.30422357,  0.99917348, -0.86242569,  0.99121445,  0.9950846 ],
       [ 0.99943239,  0.90966433,  0.18115566, -0.345191  , -0.46966954]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4